In [22]:
from agents import Agent, Runner, function_tool, gen_trace_id,trace
#from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field, EmailStr
from dotenv import load_dotenv
import gradio as gr
import asyncio
import resend
import os
from typing import Dict, List
from IPython.display import display, Markdown

In [23]:
from langchain_community.document_loaders import Docx2txtLoader, PyPDFLoader
import os

def load_document_content(file_path: str) -> str:
    ext = os.path.splitext(file_path)[-1].lower()
    
    if ext == ".pdf":
        loader = PyPDFLoader(file_path)
    elif ext == ".docx":
        loader = Docx2txtLoader(file_path)
    else:
        raise ValueError("Unsupported file type. Only .pdf and .docx are supported.")
    
    documents = loader.load()
    content = "\n".join([doc.page_content for doc in documents])
    return content



In [24]:
load_dotenv(override=True)

True

In [11]:
class Email(BaseModel):
    subject: str
    draft_email: str
    sender_email: str
    recipient_emails: List[str]

In [12]:
INSTRUCTIONS=f"""You are an assistant that drafts professional and personalized job application emails. Given the following information: \
    job title, a brief job description, sender's email, and recipient's email, generate a clear, polite, and engaging email. \
    The email should include: \
    1) A subject line relevant to the job title.\
    2) A professional greeting.\
    3) An introduction stating who the sender is and why they are contacting.\
    4) A concise paragraph highlighting the sender’s interest in the job posted and how their skills relate to the job description.\
    5) A polite call to action, inviting the recipient to respond or discuss further.\
    6) A courteous closing with the sender’s name.\
    Make sure the tone is formal and suitable for a recruiter or senior engineer.\
    Input variables you need to take from user before composing mail:\
    i) Job Title: {{job_title}}\
    ii) Job Description: {{job_description}}\
    iii) Sender Email: {{from_email}}\
    iv) Recipient Email: {{to_email}}\
    v) Resume of the user: {{Resume_details}}
    v) Write the complete email message accordingly.\
    vi) include this resume link in body itself: https://saamya2004.github.io/Saamya-Portfolio-Website/ \
    include this line in every email before writing Thanks and regards it: "Just to share a bit more: This email was created and sent by an AI-agentic email sender system I personally developed, showcasing my passion for technology and innovative problem-solving."\
    """

Email_Composition_Agent=Agent(
    name="Email composition agent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=Email
)

In [13]:
async def email_composition(job_title: str, Job_Description: str, Sender_Email: str, Recipient_Email: str, file_path: str ):

    resume_content= load_document_content(file_path)

    email_prompt = (
        f"Write an email for job title: {job_title}. "
        f"Description: {Job_Description}. "
        f"Sender email: {Sender_Email}. "
        f"Recipient email: {Recipient_Email}."
        f"resume_content:: {resume_content}"
    )

    result = await Runner.run(
        Email_Composition_Agent,
        email_prompt
    )

    return result.final_output

In [14]:
"""
@function_tool
def send_email(subject:str, html_body: str, sender: str, recipient: str)-> Dict[str,str]:
    print(Send out an email with the given subject and HTML body) 
    resend.api_key = os.environ.get('RESEND_API_KEY')
    
    params = {
       "from": sender,
        "to":[recipient],
        "subject": subject,
        "html": html_body
    }
    
    response = resend.Emails.send(params)
    return {"status": "success", "id": response.get("id")}
    """

'\n@function_tool\ndef send_email(subject:str, html_body: str, sender: str, recipient: str)-> Dict[str,str]:\n    print(Send out an email with the given subject and HTML body) \n    resend.api_key = os.environ.get(\'RESEND_API_KEY\')\n\n    params = {\n       "from": sender,\n        "to":[recipient],\n        "subject": subject,\n        "html": html_body\n    }\n\n    response = resend.Emails.send(params)\n    return {"status": "success", "id": response.get("id")}\n    '

In [ ]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
import base64
from email.mime.text import MIMEText

SCOPES = ["https://www.googleapis.com/auth/gmail.send"]

def gmail_authenticate():
    creds = None

    # token.json stores the user's access tokens
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
            creds = flow.run_local_server(port=0)

        with open("token.json", "w") as token:
            token.write(creds.to_json())

    return build("gmail", "v1", credentials=creds)


In [16]:
def create_gmail_message(sender, to, subject, html_body):
    message = MIMEText(html_body, "html")
    message["to"] = to
    message["from"] = sender
    message["subject"] = subject

    encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
    return {"raw": encoded_message}


In [17]:
@function_tool
def send_email(subject: str, html_body: str, sender: str, recipient: str) -> Dict[str, str]:
    """Send an email using Gmail API"""
    
    service = gmail_authenticate()

    email_msg = create_gmail_message(
        sender=sender,
        to=recipient,
        subject=subject,
        html_body=html_body,
    )

    sent_message = (
        service.users()
        .messages()
        .send(userId="me", body=email_msg)
        .execute()
    )

    return {"status": "success", "id": sent_message.get("id")}


In [18]:
INSTRUCTIONS ="""
You can send a nicely formatted email based on a drafted email message.
You will receive:
- Sender email
- A list of one or more recipient email addresses
- Drafted email content (subject and body)

Your task:
- Use the drafted email content as provided.
- For each recipient email, send ONE separate email with the same subject and HTML body.
"""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)

In [19]:
import json

async def process_and_send_email(email_composition_result: BaseModel):
    if hasattr(email_composition_result, 'model_dump_json'):
        email_data_str = email_composition_result.model_dump_json()
    else:
        email_data_str = json.dumps(email_composition_result.dict())
    
    result = await Runner.run(
        email_agent,
        email_data_str 
    )

    print("Email sent.")

In [20]:
async def run(job_title:str, Job_Description:str, Sender_Email: str, Recipient_Email:str, file_path: str ):
    trace_id = gen_trace_id()
    with trace("Research trace", trace_id=trace_id):
        print(f"View trace: https://platform.openai.com/traces/trace?trace_id={trace_id}")
        #yield f"View trace: https://platform.openai.com/traces/trace?trace_id={trace_id}"
        print("Starting Generating email...")
        email_formed= await email_composition(job_title, Job_Description, Sender_Email, Recipient_Email,file_path )
        print("Email is Generated")
        await process_and_send_email(email_formed)
        print("Process finished")
        
        


       
    
    

In [ ]:
result = await run("An AI & ML Engineer", "responsible for designing, developing, and deploying machine learning models and algorithms, collaborating with cross-functional teams to enhance products and services.", "Saamya<saamya.gupta21@vit.edu>","sam.gupt68@gmail.com","D:\\Sam\\ME\\Agentic_email_sender_project\\saamya_gupta_resume.pdf")

<>:1: SyntaxWarning: invalid escape sequence '\S'
C:\Users\samgu\AppData\Local\Temp\ipykernel_23752\2478216650.py:1: SyntaxWarning: invalid escape sequence '\S'
  result = await run("An AI & ML Engineer", "responsible for designing, developing, and deploying machine learning models and algorithms, collaborating with cross-functional teams to enhance products and services.", "Saamya<saamya.gupta21@vit.edu>","sam.gupt68@gmail.com","D:\Sam\ME\Agentic_email_sender_project\saamya_gupta_resume.pdf")


View trace: https://platform.openai.com/traces/trace?trace_id=trace_736d0d1dbc774e52972644c9d8284d1a
Starting Generating email...
Email is Generated
Email sent.
Process finished
